In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import random
import datetime as dt

from job_client_graphql import Client
from jupyter_cell_display import CellDisplay

### Launch
+ **JOB**: Jupyter ObservableHQ bridge  
    see. [https://jupyter-observablehq-bridge/job-server-js-graphql](TBD)

In [3]:
s = Client(host='localhost',
           port=4001,
           path='/graphql',
           channel='channel-1',
           password='mysecret',
           debug=False)

## Write

In [4]:
def build_hc_opts(data):
    hc_opts = {
        'chart': {
            'type': 'line',
            'zoomType': 'x',
        },
        'legend': {
            'enabled': False
        },
        'subtitle': {
            'text': 'Live'
        },
        'title': {
            'text': 'Data from Jupyter notebook'
        },
        'tooltip': {
            'enabled': True
        },
        'xAxis': {
            'type': 'datetime'
        },
        'yAxis': {
            'title': {
            'text': 'Value'
            }
        },
        'plotOptions': {
            'series': {
                'animation': False
            }
        },
        'series': [{
            'name': 'jupyter-data',
            'data': data
        }]
    }
    return hc_opts

def rnd():
    return random.randint(1, 10)

def to_unixtime(x):
    return x.timestamp()*1000

In [5]:
N = 30
data = []
now = dt.datetime.now()
for i in range (N):
    t = now + dt.timedelta(seconds=-N+i)
    data.append([to_unixtime(t), rnd()])

def update(x=None):
    if not x:
        x = rnd()
    print(x)
    t = dt.datetime.now()
    data.append([to_unixtime(t), x])


In [6]:
update()
hc_opts = build_hc_opts(data)
s.write('hcOpts', json.dumps(hc_opts))

7


True

## Open ObservableHQ notebook

In [8]:
url_nb = 'https://observablehq.com/d/59dd970574f9cb15'
url = url_nb+s.url_params
print(f'Open: {url}')

Open: https://observablehq.com/d/59dd970574f9cb15?host=localhost&port=4001&path=/graphql


## Subscribe

### DISPLAY

In [ ]:
out = CellDisplay(name='subscribe-status', max_lines=8)
show_lines = []

def show(channel, name, value):
    ts = dt.datetime.now().strftime('%H:%M:%S.%f')[:-4]
    s = f'received on {channel}: {name} --> {value} ({ts})'
    out.update(s, kind='text', append=True)

In [ ]:
out.display()

In [ ]:
out.clear()

### START

In [32]:
ref = s.subscribe(action=show)

### STOP

In [24]:
s.subscribe_stop(ref)

subscribe ref vnFg00 stopped


## Read

In [33]:
name = 'min-max'
s.read(name)

'{"max":1586883693845.0652,"min":1586883692717.7373}'

## Read w/ Histo

In [34]:
name = 'min-max'
s.read_histo(name, 4, timestamp=True)

[{'value': '{"max":1586883693845.0652,"min":1586883692717.7373}',
  'timestamp': datetime.datetime(2020, 4, 14, 17, 2, 9, 587000)},
 {'value': '{"max":1586883696192.2632,"min":1586883688780.502}',
  'timestamp': datetime.datetime(2020, 4, 14, 17, 2, 7, 930000)},
 {'value': '{"min":0,"max":0}',
  'timestamp': datetime.datetime(2020, 4, 14, 17, 1, 36, 557000)},
 {'value': '{"min":0,"max":0}',
  'timestamp': datetime.datetime(2020, 4, 14, 17, 1, 23, 411000)}]

## Store

In [35]:
s.store

{'channel-1': {'min-max': '{"max":1586883693845.0652,"min":1586883692717.7373}'}}